# From LDA, to naive Bayes to Logistic Regression

Author: Alexandre Gramfort

    LDA - QDA: 1h30
    Naive Bayes, logistic regression: 1h

In [ ]:
%matplotlib inline

Notations of binary classification:

- $\mathcal{Y}$ is the set of labels, here we use $\mathcal{Y} = \{-1,1\}$ in a binary classification setup,
- $\mathbf{x} = (x_1,\dots,x_p) \in \mathcal{X}\subset \mathbb{R}^p$ is an observation (a sample),
- $ \mathcal{D}_n = \{(\mathbf{x}_i , y_i), i=1,\dots n\}$ a train set
containing $n$ samples and the associated labels,
- there is a probability model which governs the generation of the data $X$ et $Y$:
$$ \forall i \in \{1,\dots,n\},  (\mathbf{x}_i , y_i) \stackrel{i.i.d }{\sim} (X,Y)$$.
- The objective is to construct from a training set $ \mathcal{D}_n $ a function
$\hat{f}:\mathcal{X} \mapsto  \{-1,1\}$ which for an unknown sample $\mathbf{x}$
(i.e. not present in the training set) can predict its label : $\hat{f}(\mathbf{x})$.

### Gaussian distribution

Gaussian density in dimension $p$, $\mathcal{N}_p(\mu, \Sigma)$ is given as :
$$
f(\mathbf{x}) = \frac{1}{(2\pi)^{p/2} \sqrt{\det(\Sigma)}} \exp\left\{ -\frac{1}{2} 
(\mathbf{x}-\mu)^\top \Sigma^{-1} (\mathbf{x}-\mu)\right\}~.
$$
where the covariance matrix of a random vector $X$ is defined as 
$\Sigma = \mathbb{E} \bigl[ (X-\mathbb{E}(X)) (X-\mathbb{E}(X))^\top\bigr]$.


#### in 1D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
a = np.random.randn(10000)
plt.hist(a, density=True, bins=40);
t = np.linspace(-5, 5, 100)
plt.plot(t, 1. / np.sqrt(2 * np.pi) * np.exp(-t**2 / 2), 'r', linewidth=6);

#### in 2D

In [ ]:
mu = [2, 2]
sigma1 = [[1, 0], [0, 1]]
sigma2 = [[4, 0], [0, 1]]
sigma3 = [[1, .8], [.8, 1]]

X = np.random.multivariate_normal(mu, sigma1, size=2000)
print(X.shape)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 5))

for ax, sigma in zip(axes, [sigma1, sigma2, sigma3]):
    X = np.random.multivariate_normal(mu, sigma, size=2000)
    ax.scatter(X[:, 0], X[:, 1])
    ax.axis('equal')
    ax.set_xlim([-5, 10])
    ax.set_ylim([-5, 10])

## Generative models

Conditional probability:

$$
f_k(x) = \mathbb{P}\{x | y = k\}
$$

Probability of being in class k:
$$
\mathbb{P}\{y = k\} = \pi_k
$$

Mixture model:
$$
\mathbb{P}\{x\} = \sum_{k \in \{-1, 1\}} \pi_k f_k(x)
$$

Bayes' rule:
$$
\mathbb{P}\{y=k | x\} = \frac{\mathbb{P}\{y = k\} \mathbb{P}\{x | y = k\}}{\mathbb{P}\{x\}} = \frac{\pi_k f_k(x)}{\sum_{k' \in \{-1, 1\}} \pi_{k'} f_{k'}(x)}
$$

### LDA (Linear Discriminant Analysis)

When using a linear discriminant analysis (LDA) we assume:

$$
\mathbb{P}\{x | y=1\} = \mathcal{N}_p(\mu_1, \Sigma)
$$

$$
\mathbb{P}\{x | y=-1\} = \mathcal{N}_p(\mu_{-1}, \Sigma)
$$

i.e. the conditional probability are Gaussian with **same covariance** but **different centers** for each class.

#### Example:

In [ ]:
mu1 = [2, 2]
mu2 = [-2, -3]
sigma = [[1, 0], [0, 1]]

X1 = np.random.multivariate_normal(mu1, sigma, size=2000)
X2 = np.random.multivariate_normal(mu2, sigma, size=2000)

plt.scatter(X1[:, 0], X1[:, 1], color='b')
plt.scatter(X2[:, 0], X2[:, 1], color='g');
plt.axis('equal');

Log ratio:

$$
\log \left(\frac{\mathbb{P}\{Y=+1 \mid X=\mathbf{x}\}}{\mathbb{P}\{Y=-1 \mid X=\mathbf{x}\}}\right)
= x^T \Sigma^{-1} (\mu_{1} - \mu_{-1}) + \frac{1}{2} (\mu_{1}^T \Sigma^{-1} \mu_{1} - \mu_{-1}^T \Sigma^{-1} \mu_{-1}) + \log(\frac{\pi_{1}}{\pi_{-1}})
$$

Decision function:

$$
x^T \Sigma^{-1} (\mu_{1} - \mu_{-1}) + \frac{1}{2} (\mu_{1}^T \Sigma^{-1} \mu_{1} - \mu_{-1}^T \Sigma^{-1} \mu_{-1}) + \log(\frac{\pi_{1}}{\pi_{-1}}) > 0 \Rightarrow y = 1
$$

It is a **linear** function of the features !

### Example

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def demo_clf(clf, X, y, proba=False):
    clf.fit(X, y)

    h = .02  # step size in the mesh
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    if proba:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    else:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(1, figsize=(4, 3))
    cmap = plt.cm.bwr
    plt.pcolormesh(xx, yy, Z, cmap=cmap, clim=[0, 1], alpha=0.5)
    plt.clim([0, 1])
    plt.colorbar()

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=cmap)

    plt.axis('square')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())

X = np.concatenate((X1, X2), axis=0)
y = np.array([1] * len(X1) + [-1] * len(X2))

demo_clf(LinearDiscriminantAnalysis(), X, y, proba=True)

In [ ]:
clf = LinearDiscriminantAnalysis()
clf.fit(X, y)

In [ ]:
clf.coef_, clf.intercept_

## QDA (Quadratic discriminant analysis)

When using a quadratic discriminant analysis (QDA):

$$
\mathbb{P}\{x | y=1\} = \mathcal{N}_p(\mu_1, \Sigma_{1})
$$

$$
\mathbb{P}\{x | y=-1\} = \mathcal{N}_p(\mu_{-1}, \Sigma_{-1})
$$

i.e. different covariances with different centers for each class.

as a consequence with have a **quadratic** boundary.

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

mu1 = [2, 2]
mu2 = [-2, -3]
sigma1 = [[1, 0], [0, 1]]
sigma2 = [[1, 0.8], [0.8, 1]]

X1 = np.random.multivariate_normal(mu1, sigma1, size=2000)
X2 = np.random.multivariate_normal(mu2, sigma2, size=2000)
X = np.r_[X1, X2]
y = np.array([1] * len(X1) + [-1] * len(X2))

demo_clf(QuadraticDiscriminantAnalysis(), X, y, proba=True)

# Naive Bayes

Naive Bayes is also a **generative** model. It however assumes that all the features are independent conditionnaly on $y$.

$$
\mathbb{P}\{x | y=k\} = \prod_{j=1}^p \mathbb{P}\{x_j | y=k\}
$$

### Gaussian Naive Bayes

$$
\mathbb{P}\{x_j | y=k\} = \frac{1}{(2\pi)^{1/2} \sigma_j^k} \exp \left\{ -\frac{
(x_j-\mu_j^k)^2}{2 (\sigma_j^k)^2}\right\}~.
$$

As the variance parameters dependent on the class we have a quadractic boundary condition.

In [ ]:
from sklearn.naive_bayes import GaussianNB

mu1 = [2, 2]
mu2 = [-2, -3]
sigma1 = [[0.3, -0.2], [-0.2, 1]]
sigma2 = [[1, 0.8], [0.8, 1]]

X1 = np.random.multivariate_normal(mu1, sigma1, size=2000)
X2 = np.random.multivariate_normal(mu2, sigma2, size=2000)
X = np.r_[X1, X2]
y = np.array([1] * len(X1) + [-1] * len(X2))

demo_clf(GaussianNB(), X, y, proba=True)

#### Question : Can I estimate Naive Bayes with streaming data?

## Logistic Regression

Logistic regression is a **discriminative** classification approach (although it's called regression...)

It follows the model of LDA with a log ratio that is a linear function of the features:

$$
\log \left(\frac{\mathbb{P}\{Y=+1 \mid X=\mathbf{x}\}}{\mathbb{P}\{Y=-1 \mid X=\mathbf{x}\}}\right)
= x^T \beta + \beta_0
$$

Decision function:

$$
x^T \beta + \beta_0 > 0 \Rightarrow y = 1
$$

It is a **linear** function of the features !

We then can get the conditional probabilities:

$$
\mathbb{P}\{Y=1 \mid X=\mathbf{x}\} = \frac{\exp(\mathbf{x}^T \beta + \beta_0)}{1 + \exp(\mathbf{x}^T \beta + \beta_0)}
$$

$$
\mathbb{P}\{Y=-1 \mid X=\mathbf{x}\} = \frac{1}{1 + \exp(\mathbf{x}^T \beta + \beta_0)}
$$

In practice $\beta$ and $\beta_0$ are computed by maximizing the likelihood of the training data under this model. It reads:

$$
\hat{\beta}, \hat{\beta}_0 = \mathrm{argmax}_{\beta, \beta_0} \sum_{i=1}^n \sum_k 1_{\{Y_i = k\}} \log (\mathbb{P}\{Y=k \mid X=\mathbf{x}_i, \beta, \beta_0 \})
$$

One can show that it leads with y=1 or y=-1 to:

$$
\hat{\beta}, \hat{\beta}_0 = \mathrm{argmin}_{\beta, \beta_0} \sum_{i=1}^n \log \{1 + \exp(-y_i(\mathbf{x}_i^T \beta + \beta_0) \})
$$

In [ ]:
from sklearn.linear_model import LogisticRegression

mu1 = [2, 2]
mu2 = [-2, -3]
sigma1 = [[0.3, -0.2], [-0.2, 1]]
sigma2 = [[1, 0.8], [0.8, 1]]

X1 = np.random.multivariate_normal(mu1, sigma1, size=2000)
X2 = np.random.multivariate_normal(mu2, sigma2, size=2000)
X = np.r_[X1, X2]
y = np.array([1] * len(X1) + [-1] * len(X2))

demo_clf(LogisticRegression(solver='lbfgs'), X, y, proba=True)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

def demo_clf(clf, X, y, proba=False):
    clf.fit(PolynomialFeatures(2).fit_transform(X), y)

    h = .02  # step size in the mesh
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    XX = PolynomialFeatures(2).fit_transform(np.c_[xx.ravel(), yy.ravel()])
    if proba:
        Z = clf.predict_proba(XX)[:, 1]
    else:
        Z = clf.predict(XX)

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(1, figsize=(4, 3))
    cmap = plt.cm.bwr
    plt.pcolormesh(xx, yy, Z, cmap=cmap, clim=[0, 1])
    plt.colorbar()

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=cmap)

    plt.axis('square')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())

demo_clf(LogisticRegression(solver='lbfgs'), X, y, proba=True)